In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
import warnings
warnings.simplefilter('ignore')

import logging
logging.getLogger('statsforecast').setLevel(logging.ERROR)

# StatsForecast ⚡️

> StatsForecast offers a collection of popular univariate time series forecasting models optimized for high performance and scalability. 

## Installation

You can install the *released version* of `StatsForecast` from the [Python package index](https://pypi.org) with:

```python
pip install statsforecast
```
We are constantly updating StatsForecast, so we suggest fixing the version to avoid issues. `pip install statsforecast=="1.0.0"`


We recommend installing your libraries inside a python virtual or conda environment. To install the latest *released version* of `StatsForecast` from [conda](https://anaconda.org) run this command:

```python
conda install -c conda-forge statsforecast
```



## Get Started

To get started with a minimal example, follow this [guide](./examples/Getting_Started_short.ipynb)

Or follow this [stept by step guide ](./examples/Getting_Started_complete.ipynb) to train, evaluate and select multiple models on many time series. 

## Features

* Fastest and most accurate implementations of `AutoARIMA`, `ETS`, `AutoCES`, and `MSTL` in Python. 
* Out-of-the-box compatibility with Spark, Dask, and Ray.
* Probabilistic Forecasting and Confidence Intervals.
* Anomaly Detection.
* Familiar sklearn syntax: `.fit` and `.predict`. 
* Intermittent demand forecasting with `CrostonClassic`, `CrostonSBA`, `CrostonOptimized`, `ADIDA`, and `IMAPA`.
* Benchmarking at scale with models like `HistoricAverage`, `SeasonalWindowAverage`, `SeasonalNaive`, `Naive`, `RandomWalkWithDrift`, `WindowAverage`, `SeasonalExponentialSmoothing`, and `TSB`.



## Highlights

* Inclusion of `exogenous variables` and `prediction intervals` for ARIMA.
* 20x faster than `pmdarima`.
* 1.5x faster than `R`.
* 500x faster than `Prophet`. 
* 4x faster than `statsmodels`.
* Compiled to high performance machine code through [`numba`](https://numba.pydata.org/).
* 1,000,000 series in [30 min](https://github.com/Nixtla/statsforecast/tree/main/experiments/ray) with [ray](https://github.com/ray-project/ray).
* Replace FB-Prophet in two lines of code and gain speed and accuracy. Check the experiments [here](https://github.com/Nixtla/statsforecast/tree/main/experiments/arima_prophet_adapter).


Missing something? Please open an issue or write us in [![Slack](https://img.shields.io/badge/Slack-4A154B?&logo=slack&logoColor=white)](https://join.slack.com/t/nixtlaworkspace/shared_invite/zt-135dssye9-fWTzMpv2WBthq8NK0Yvu6A)


## Why? 

Current Python alternatives for statistical models are slow, inaccurate and don't scale well. So we created a library that can be used to forecast in production environments or as benchmarks.  `StatsForecast` includes an extensive battery of models that can efficiently fit millions of time series.

## Accuracy & Speed 

### ARIMA 
The `AutoARIMA` model implemented in `StatsForecast` is **20x faster** than `pmdarima` and **1.5x faster** than `R`  while improving accuracy. You can see the exact comparison and reproduce the results [here](./experiments/arima/).

### ETS

StatsForecast's `ETS` is **4x faster than StatsModels'** and **1.6x faster than R's**, with improved accuracy and robustness. You can see the exact comparison and reproduce the results[here](./experiments/ets/)

### Benchmarks at Scale

With `StatsForecast` you can fit 9 benchmark models on **1,000,000** series in under **5 min**. Reproduce the results [here](./experiments/benchmarks_at_scale/). 



## Getting Started 
You can run this notebooks to get you started. 

* Example of different `AutoARIMA` models on M4 data [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nixtla/statsforecast/blob/main/nbs/examples/AutoArima_vs_Prophet.ipynb)  
    * In this notebook we present Nixtla's `AutoARIMA`. 
    The `AutoARIMA` model is widely used to forecast time series in production and as a benchmark. However, the alternative python implementation (`pmdarima`) is so slow that prevents data scientists from quickly iterating and deploying `AutoARIMA` in production for a large number of time series. 

* Shorter Example of fitting and `AutoARIMA` and an `ETS` model. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nixtla/statsforecast/blob/main/nbs/examples/Getting_Started_with_Auto_Arima_and_ETS.ipynb)


* Benchmarking 9 models on millions of [series](./experiments/benchmarks_at_scale/).

## How to contribute
See [CONTRIBUTING.md](https://github.com/Nixtla/statsforecast/blob/main/CONTRIBUTING.md).

## References

*  The `AutoARIMA` model is based (translated) from the R implementation included in the [forecast](https://github.com/robjhyndman/forecast) package developed by Rob Hyndman.
*  The `ETS` model is based (translated) from the R implementation included in the [forecast](https://github.com/robjhyndman/forecast) package developed by Rob Hyndman.